In [1]:
import torch
import numpy as np
from spyrit.misc.walsh_hadamard import walsh_matrix
from spyrit.core.Forward_Operator import *
from spyrit.core.Preprocess import *

## Instantiate Hsub

In [2]:
h,w = 32, 32
img_size = h*w
nb_measurements = 400
batch_size = 10
Hcomplete = walsh_matrix(img_size)
Hsub = Hcomplete[0:nb_measurements,:]
Perm = np.random.random([w*h,w*h])

## Instantiate Forward_operator and Forward_operator_Split

In [3]:
FO = Forward_operator(Hsub)
FO_Split = Forward_operator_Split(Hsub)
FO_Split_ft_had = Forward_operator_Split_ft_had(Hsub, Perm, w, h)

## Split_diag_poisson_preprocess

In [4]:
alpha = 9
SPP = Preprocess_Split_diag_poisson(alpha, nb_measurements, img_size)

### forward

In [5]:
x = torch.tensor(np.random.random([batch_size,img_size]), dtype=torch.float)
y_FO_Split = FO_Split(x)
y_SPP = SPP(y_FO_Split, FO_Split)
print(y_FO_Split.shape)
print(y_SPP.shape)

torch.Size([10, 800])
torch.Size([10, 400])


### sigma

In [6]:
y_sigma = SPP.sigma(y_FO_Split)
print(y_sigma.shape)

torch.Size([10, 400])


### sigma_expe

In [7]:
SPP.set_expe(gain=1, mudark=0, sigdark=0, nbin=1)
y_sigma_expe = SPP.sigma_expe(x)
print(y_sigma_expe.shape)

torch.Size([10, 400])


### sigma_from_image

In [8]:
y = SPP.sigma_from_image(x, FO)

IndexError: index 400 is out of bounds for dimension 0 with size 400

### forward_expe

In [9]:
xsub = torch.tensor(np.random.random([10, 2*400]), dtype=torch.float)
y_FE, alpha_est = SPP.forward_expe(xsub, FO_Split_ft_had)
print(y_FE[0].shape)
print(alpha_est)

torch.Size([400])
tensor([0.0221, 0.0242, 0.0210, 0.0224, 0.0214, 0.0220, 0.0206, 0.0210, 0.0274,
        0.0248])


### denormalize_expe

In [15]:
norm = torch.tensor(9*np.ones(10), dtype=float)
y_DE = SPP.denormalize_expe(x, norm, 32, 32)

> /home/fadoua/ARIM_CREATIS/clone-spyrit/spyrit/spyrit/core/Preprocess.py(201)denormalize_expe()
    199 
    200         # Denormalization
--> 201         norm = norm.view(bc,1,1,1)
    202         norm = norm.expand(bc,1,h,w)
    203         x = (x+1)/2*norm



ipdb>  c


RuntimeError: The size of tensor a (1024) must match the size of tensor b (32) at non-singleton dimension 3

## Preprocess_shift_poisson

In [ ]:
PSP = Preprocess_shift_poisson(alpha, nb_measurements, img_size)

### forward